## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Ushuaia,AR,-54.8000,-68.3000,38.86,81,75,5.75,broken clouds
1,1,Passy,FR,45.9203,6.6877,68.49,71,65,3.80,broken clouds
2,2,Carnarvon,AU,-24.8667,113.6333,71.02,70,100,9.98,overcast clouds
3,3,Torbay,CA,47.6666,-52.7314,55.27,93,100,9.22,overcast clouds
4,4,Namie,JP,37.4833,141.0000,96.71,82,64,4.32,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
8,8,Bathsheba,BB,13.2167,-59.5167,81.21,84,75,11.50,broken clouds
18,18,Carutapera,BR,-1.1950,-46.0200,75.70,90,7,4.68,clear sky
20,20,Axim,GH,4.8699,-2.2405,75.20,74,98,11.39,overcast clouds
28,28,Vostok,RU,46.4856,135.8833,87.64,43,100,4.16,overcast clouds
31,31,Kapaa,US,22.0752,-159.3190,80.58,79,20,9.22,few clouds
34,34,Samarai,PG,-10.6167,150.6667,79.18,85,69,14.56,light rain
40,40,Nukus,UZ,42.4531,59.6103,80.58,44,20,11.50,few clouds
43,43,Jacksonville,US,30.3322,-81.6556,81.01,85,0,4.61,clear sky
44,44,Wanning,CN,18.8003,110.3967,90.21,61,12,7.25,few clouds
49,49,Juana Diaz,PR,18.0525,-66.5066,77.59,81,1,4.99,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# preferred_cities_df.isnull().drop()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
8,Bathsheba,BB,81.21,broken clouds,13.2167,-59.5167,
18,Carutapera,BR,75.70,clear sky,-1.1950,-46.0200,
20,Axim,GH,75.20,overcast clouds,4.8699,-2.2405,
28,Vostok,RU,87.64,overcast clouds,46.4856,135.8833,
31,Kapaa,US,80.58,few clouds,22.0752,-159.3190,
34,Samarai,PG,79.18,light rain,-10.6167,150.6667,
40,Nukus,UZ,80.58,few clouds,42.4531,59.6103,
43,Jacksonville,US,81.01,clear sky,30.3322,-81.6556,
44,Wanning,CN,90.21,few clouds,18.8003,110.3967,
49,Juana Diaz,PR,77.59,clear sky,18.0525,-66.5066,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()
hotel_df


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
8,Bathsheba,BB,81.21,broken clouds,13.2167,-59.5167,Atlantis Hotel
18,Carutapera,BR,75.70,clear sky,-1.1950,-46.0200,Lidera
20,Axim,GH,75.20,overcast clouds,4.8699,-2.2405,Axim Beach Resort And Spa
28,Vostok,RU,87.64,overcast clouds,46.4856,135.8833,
31,Kapaa,US,80.58,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
672,Nhulunbuy,AU,79.41,broken clouds,-12.2333,136.7667,East Arnhem Real Estate
678,Kavieng,PG,83.62,broken clouds,-2.5744,150.7967,Nusa Island Retreat
681,Naze,JP,79.72,moderate rain,28.3667,129.4833,Shishime Hotel
682,Kendari,ID,83.21,broken clouds,-3.9450,122.4989,Hotel Wixel


In [11]:
clean_hotel_df = hotel_df
nan_value = float("NaN")
clean_hotel_df.replace("",nan_value,inplace=True)

clean_hotel_df.dropna(subset = ["Hotel Name"], inplace = True)
# clean_hotel_df

In [12]:
# 8a. Create the output File (CSV)
output_data_file ="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Weather Description}
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
# well = fig.savefig("WeatherPy_vacation_map.png")
fig



Figure(layout=FigureLayout(height='420px'))